In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [5]:
path_cdr='/storage2/tkyeimiah/CDR/'

In [6]:
CT_combined_data = []
time_coords = []

for yyyy in range(1990, 2021):
    for mm in range(1, 13):
        pattern = f"{path_cdr}seaice_conc_monthly_nh_{yyyy}{mm:02d}_*_v04r00.nc"
        file_paths = glob.glob(pattern)
        
        monthly_datasets = []
        for file_path in file_paths:
            ds = xr.open_dataset(file_path)
            CT = ds['cdr_seaice_conc_monthly']
            monthly_datasets.append(CT)
        
        # Assuming you want to combine all monthly data in each year before appending
        if monthly_datasets:
            combined_CT = xr.concat(monthly_datasets, dim='tdim')
            CT_combined_data.append(combined_CT)
            
            # Create a time coordinate for each month
            time_coords.append(pd.Timestamp(year=yyyy, month=mm, day=1))

# Now combine all monthly data into one dataset
CT_all_combined = xr.concat(CT_combined_data, dim='tdim')


# Assign time coordinates
#CT_all_combined = CT_all_combined.assign_coords(tdim=("tdim", time_coords))
CT_all_combined = CT_all_combined.assign_coords(tdim=("tdim", time_coords)).drop_vars('time')

CT_all_combined.to_netcdf('/storage2/tkyeimiah/CDR/CDR_CT.nc')